In [ ]:
import csv
from scipy import cluster as cluster
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import OrderedDict

# import pdb

In [ ]:
x = np.random.uniform(low=0, high=360, size=20).astype(int)
x

array([246, 225,  44, 247, 277, 124,  29,  27,  49,  31,  39, 224,  25,
       344, 358, 162,  57, 170, 101, 179])

In [ ]:
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")
base_path = './drive/MyDrive/Colab Notebooks/Data/AI/hw4/'

Mounted at /content/drive


In [ ]:
def load_data(filepath):
  with open(filepath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = []
    for row in reader:
      row = dict(row)
      del row['Generation']
      del row['Legendary']
      data.append(row)
      
    data = data[:20]
    for pokemon in data:
      for key, val in pokemon.items():
        if key not in ['Type 1', 'Type 2', 'Name']:
          pokemon[key] = int(val)
    return data

In [ ]:
data = load_data(base_path + 'Pokemon.csv')

In [ ]:
def calculate_x_y(stats):
  x = stats['Attack'] + stats['Sp. Atk'] + stats['Speed'] 
  y = stats['Defense'] + stats['Sp. Def'] + stats['HP']

  return (x,y)

In [ ]:
data_s = []
for d in data:
  data_s.append(calculate_x_y(d))
data_s

In [ ]:
def calEucD(d1,d2):
  return ( math.sqrt((d1[0] - d2[0])**2 + (d1[1] - d2[1])**2))

In [ ]:
def candidate_clusters(dataset):
  alldist = []
  for i in dataset:
    for j in dataset:
      if( i != j):
        #for more datapoints in i and j clusters
        min_d = float('inf')
        min_d_points = []
        for all_i_clust in dataset[i]:
          for all_j_clust in dataset[j]:
            d = calEucD(all_i_clust, all_j_clust)
            if d < min_d:
              min_d = d
              min_d_points = [all_i_clust, all_j_clust]
        if ([min_d, sorted(tuple([i,j]))]) not in alldist:
          alldist.append([min_d, sorted(tuple([i,j])), min_d_points])
  # print(alldist)
  alldist.sort(key = lambda x: (x[0], x[1][0], x[1][1]))
  return alldist[0]

In [ ]:
def hac(dataset, display = False):
  cleaned_data = OrderedDict()
  plt.figure()
  if display:
    xs_ = []
    ys_ = []
  i = 0
  for items in dataset:
    if items[0] in [np.nan, np.inf] or items[1] in [np.nan, np.inf]:
      pass
    else:
      cleaned_data[i] = tuple([(items)])
      i += 1
      if display:
        xs_.append(items[0])
        ys_.append(items[1])
  if display:
    plt.scatter(xs_, ys_)
  z = []
  m = len(cleaned_data)
  for i in range(0, len(cleaned_data) -1):
    c_c = candidate_clusters(cleaned_data)
    # print(c_c)
    index = m+i  
    cleaned_data[index] = cleaned_data[c_c[1][0]] + cleaned_data[c_c[1][1]]
    if display:
      xs = []
      ys = []
      for item in c_c[2]:
        xs.append(item[0])
        ys.append(item[1])
      for item in c_c[2]:
        xs.append(item[0])
        ys.append(item[1])
      plt.scatter(xs_,ys_)
      plt.plot(xs,ys)
      if( i == m-2):
        plt.pause(10)
      plt.pause(0.2)
      # if(i <= m-3):
      #   plt.close()
    del cleaned_data[c_c[1][0]]
    del cleaned_data[c_c[1][1]]
    # print(cleaned_data)

    z.append([c_c[1][0], c_c[1][1], c_c[0], len(cleaned_data[m+i])])
  return np.array(z)

In [ ]:
# res = cluster.hierarchy.linkage(data_s)
# res
res = hac(data_s)
res

In [ ]:
def random_x_y(m):
  x = np.random.uniform(low=0, high=360, size=m)
  y = np.random.uniform(low=0, high=360, size=m)

  data_x_y = []
  for i in range(m):
    data_x_y.append((x[i], y[i]))
  return data_x_y



In [ ]:
data_xy = random_x_y(150)
data_xy

In [ ]:
def imshow_hac(dataset):
  return(hac(dataset, True))

In [ ]:
res = cluster.hierarchy.linkage(data_xy)
res

In [ ]:
res = hac(data_xy)
res

In [ ]:
res = hac([calculate_x_y(stats) for stats in load_data(base_path + 'Pokemon.csv')])
res